Aluna: Samantha Pinheiro Gomes

RM: 362698

Turma: 29ABD

Matéria: Distributed Data Processing & Storage

Este notebook realiza a análise do arquivo "results", que contém dados de diversos jogos realizados em várias cidades e países ao redor do mundo, incluindo as datas dos eventos e seus respectivos placares. 



Import de bibliotecas

In [0]:
import pandas as pd
import requests
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, BooleanType
from io import StringIO
from pyspark.sql.functions import to_date, col, max

Baixa arquivo armazenado no GitHub

In [0]:
## URL do arquivo no GitHub 

url = "https://raw.githubusercontent.com/samppinheiro/fiap_distributed_data_processing_-_storage/main/results.csv"

## Armazena os dados 
response = requests.get(url)
csv_content = response.content.decode("utf-8")

## Lê com pandas
pandas_df = pd.read_csv(StringIO(csv_content))



Load do dados em um DF do spark

In [0]:
## Define schema do dataframe 

schema = StructType([
    StructField("date", StringType(), True),
    StructField("home_team", StringType(), True),
    StructField("away_team", StringType(), True),
    StructField("home_score", IntegerType(), True),
    StructField("away_score", IntegerType(), True),
    StructField("tournament", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("neutral", BooleanType(), True)
])

## Converte para Spark DataFrame
df_results = spark.createDataFrame(pandas_df, schema=schema)

## Converte o formato da coluna "date" 
df_results = df_results.withColumn("date", to_date("date", "yyyy-MM-dd"))



Define dataframe para iniciar a análise

In [0]:
df_results_consol = (
    df_results
    .withColumnRenamed("date", "dt_torneio")
    .withColumnRenamed("home_team", "nm_time_casa")
    .withColumnRenamed("away_team", "nm_time_visitante")
    .withColumnRenamed("home_score", "nr_placar_casa")
    .withColumnRenamed("away_score", "nr_placar_visitante")
    .withColumnRenamed("tournament", "nm_torneio")
    .withColumnRenamed("city", "nm_cidade")
    .withColumnRenamed("country", "nm_pais")    
).drop("neutral")

Análises realizadas no arquivo

In [0]:
## Quantos registros existem na base?? 

nm_registro = df_results_consol.count()

display(f"Existem {nm_registro} registros na base")

In [0]:
## Quantas equipes únicas mandantes existem na base?

nm_equipes_mandantes = df_results_consol.select("nm_time_casa").distinct().count()

display(f"Existem {nm_equipes_mandantes} equipes mandantes na base")

In [0]:
## Quantas vezes as equipes mandantes saíram vitoriosas?

nm_vitorias_mandantes = (df_results_consol
                         .filter(col("nr_placar_casa") > col("nr_placar_visitante"))
                         .count())

display(f"As equipes mandantes saíram vitoriosas {nm_vitorias_mandantes} vezes")

In [0]:
## Quantas vezes as equipes visitantes saíram vitoriosas?

nm_vitorias_visitantes = (df_results_consol
                         .filter(col("nr_placar_visitante") > col("nr_placar_casa"))
                         .count())

display(f"As equipes visitantes saíram vitoriosas {nm_vitorias_visitantes} vezes")

In [0]:
## Quantas partidas resultaram em empate?
nm_empate = (df_results_consol
                         .filter(col("nr_placar_casa") == col("nr_placar_visitante"))
                         .count())

display(f'{nm_empate} partidas resultaram em empate')

In [0]:
## Quantas partidas foram realizadas em cada país?

df_pais = (df_results_consol
           .groupBy("nm_pais")
           .count()
           .orderBy(col("count").desc())

           .withColumnRenamed("count", "qt_partidas")

)
display(df_pais)


In [0]:
## Qual país teve mais partidas?

df_partidas = df_pais.orderBy(col("qt_partidas").desc()).limit(1)
display(df_partidas)

In [0]:
## Qual a partida com maior número de gols?

max_nr_gols = (df_results_consol
               .withColumn("nr_maior_placar", col("nr_placar_casa") + col("nr_placar_visitante"))
               .orderBy(col("nr_maior_placar").desc())
               .select("dt_torneio", "nm_time_casa", "nm_time_visitante", "nr_maior_placar")
               .limit(1)
)
display(max_nr_gols)

In [0]:
## Qual a maior goleada?
max_diff_gols = (df_results_consol
               .withColumn("nr_maior_diff_gols", col("nr_placar_casa") - col("nr_placar_visitante"))
               .orderBy(col("nr_maior_diff_gols").desc())
               .select('*')
               .limit(1)
)
display(max_diff_gols)

In [0]:
## Quantos jogos ocorreram no Brasil?

df_brasil = (df_results_consol
             .filter(col("nm_pais") == "Brazil")
             .count()                          
             )

display(f"{df_brasil} jogos foram realizados no Brasil")